In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_drug6_KY_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv").sort_values('person_id')
x = data.loc[:,:'current_violence20']
y = data['drug_six_month'].values

In [3]:
#### CART
depth = [8,9,10]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         seed=816)

#### EBM
estimators = [40]
depth = [2]
learning_rate = [0.1]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate=learning_rate,
                       depth=depth,
                       estimators=estimators,
                       seed =816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binh

In [4]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.7694426520036945, 0.030450052937010308)

In [5]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.7767213806183678, 0.00857705708791905)

## Lasso Stumps

In [6]:
## load whole data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on='person_id')
X_stumps, Y_stumps = data.loc[:,:'current_violence201'], data['drug_six_month'].values
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_train_stumps.csv")
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_test_stumps.csv")
X_train_stumps, Y_train_stumps = train_stumps.loc[:,:'current_violence201'], train_stumps['drug_six_month'].values
X_test_stumps, Y_test_stumps = test_stumps.loc[:,:'current_violence201'], test_stumps['drug_six_month'].values
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

##### Single Stump Model

In [9]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.002, 
                                      columns=cols, 
                                      seed=816)
len(single_stump_model['features'])

18

In [10]:
stumps.latex_stump_table(single_stump_model['coefs'], 
                   single_stump_model['features'], 
                   single_stump_model['intercept'], 
                   single_stump_model['dictionary'])

\begin{{tabular}}{{|l|r|r|}} \hline
1. age_at_current_charge35 & 0.08299238201045521 & +... \\ \hline
2. age_at_current_charge39 & 0.004269935346527674 & +... \\ \hline
3. p_arrest2 & 0.5247739792221712 & +... \\ \hline
4. p_arrest3 & 0.2793362628173456 & +... \\ \hline
5. p_arrest4 & 0.10542279230638242 & +... \\ \hline
6. p_arrest5 & 0.023162851124606063 & +... \\ \hline
7. p_arrest6 & 0.05089481665534015 & +... \\ \hline
8. p_violence1 & 0.05000383393694661 & +... \\ \hline
9. p_felony1 & 0.5127888352247083 & +... \\ \hline
10. p_drug1 & 0.8915472969346026 & +... \\ \hline
11. p_drug2 & 0.1481396314445215 & +... \\ \hline
12. p_drug3 & 0.25155137884091455 & +... \\ \hline
13. ADE1 & 0.6813652171051814 & -... \\ \hline
14. p_pending_charge1 & 0.32469947117084297 & +... \\ \hline
15. p_probation1 & 0.11009442854177483 & +... \\ \hline
16. p_incarceration1 & 0.3083656740843957 & +... \\ \hline
17. five_year1 & 0.24336186434492058 & -... \\ \hline
18. current_pending_charge1 & 0.1562539

#### Nested Cross Validation

In [7]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.001, 0.002]}, 
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [8]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}],
 0.7768786226998354,
 0.0008030259055355815)

## RiskSLIM

In [11]:
## load whole data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv")
original_data = original_data.loc[:,['person_id', 'age_at_current_charge', 'p_charges']]

## load train & test data
train_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_train_stumps.csv")
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_test_stumps.csv")

In [12]:
best_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.001, 
                                      columns=cols, 
                                      seed=816)
len(best_stump_model['features'])

14

In [13]:
selected_features = best_stump_model['features']

In [14]:
### Subset features
if 'sex1' in selected_features:
    selected_features = ['drug_six_month', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['drug_six_month', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on='person_id')
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [15]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator = indicator,
                                       y_label='drug_six_month', 
                                       max_coef=5, 
                                       max_coef_number=5, 
                                       max_runtime=1000,
                                       max_offset=100,
                                       c=1e-3, 
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/09/19 @ 10:57 AM | 193 rows in lookup table
11/09/19 @ 10:57 AM | ------------------------------------------------------------
11/09/19 @ 10:57 AM | runnning initialization procedure
11/09/19 @ 10:57 AM | ------------------------------------------------------------
11/09/19 @ 10:57 AM | CPA produced 2 cuts
11/09/19 @ 10:57 AM | running naive rounding on 6 solutions
11/09/19 @ 10:57 AM | best objective value: 0.5797
11/09/19 @ 10:57 AM | rounding produced 5 integer solutions
11/09/19 @ 10:57 AM | best objective value is 0.5598
11/09/19 @ 10:57 AM | running sequential rounding on 6 solutions
11/09/19 @ 10:57 AM | best objective value: 0.5797
11/09/19 @ 10:57 AM | sequential rounding produced 3 integer solutions
11/09/19 @ 10:57 AM | best objective value: 0.2769
11/09/19 @ 10:57 AM | polishing 8 solutions
11/09/19 @ 10:57 AM | best objective value: 0.2769
11/09/19 @ 10:57 AM | polishing produced 5 integer solutions
11/09/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/09/19 @ 10:57 AM | adding 257 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1680.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/09/19 @ 10:58 AM | adding 262 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1718.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.1549244713664879
CPXPARAM_MIP_Tolerances_UpperCutoff              0.16775973726385068


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/09/19 @ 10:58 AM | adding 296 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1678.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/09/19 @ 11:18 AM | adding 254 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1711.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

11/09/19 @ 11:19 AM | ------------------------------------------------------------
11/09/19 @ 11:19 AM | switching loss computation from lookup to weighted
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.15647567550073788
CPXPARAM_MIP_Tolerances_UpperCutoff              0.16259246648202899


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/09/19 @ 11:19 AM | adding 253 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1626.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/09/19 @ 11:20 AM | adding 270 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1716.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [16]:
np.mean(riskslim_summary['validation_auc']), np.mean(riskslim_summary['test_auc'])

(0.7517979314874838, 0.7517732771934686)

### Single RiskSLIM Model

In [17]:
selected_features = ["drug_six_month"] + best_stump_model['features']
sub_train_data = train_data[selected_features]
sub_test_data = test_data[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [18]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': cols,
    'outcome_name': 'drug_six_month',
    'sample_weights': sample_weights
}

In [19]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-3, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
11/09/19 @ 11:21 AM | 193 rows in lookup table
11/09/19 @ 11:21 AM | ------------------------------------------------------------
11/09/19 @ 11:21 AM | runnning initialization procedure
11/09/19 @ 11:21 AM | ------------------------------------------------------------
11/09/19 @ 11:21 AM | CPA produced 2 cuts
11/09/19 @ 11:21 AM | running naive rounding on 6 solutions
11/09/19 @ 11:21 AM | best objective value: 0.5868
11/09/19 @ 11:21 AM | rounding produced 5 integer solutions
11/09/19 @ 11:21 AM | best objective value is 0.5734
11/09/19 @ 11:21 AM | running sequential rounding on 6 solutions
11/09/19 @ 11:21 AM | best objective value: 0.5868
11/09/19 @ 11:21 AM | sequential rounding produced 3 integer solutions
11/09/19 @ 11:21 AM | best objective value: 0.2760
11/09/19 @ 11:21 AM | polishing 8 solutions
11/09/19 @ 11:21 AM | best objective value: 0.2760
11/09/19 @ 11:21 AM | polishing produced 5 integer solutions
11/09/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/09/19 @ 11:21 AM | adding 275 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1640.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [20]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [21]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.7531446766484897

In [22]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.7405516361591691

### Arnold PSA

In [23]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    data = data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    data[score] = data[score + "_raw"]
    data = data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        data[score] = data[score].astype(int)
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['drug_six_month'].values

In [24]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
arnold_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    arnold_auc.append(roc_auc_score(test_y, test_x['arnold_nca'].values))
    
    ################################
    ## race_auc
    arnold_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                               probs = test_x['arnold_nca'],
                                               labels = test_y)
    arnold_race_auc_final = arnold_race_auc.assign(fold_num = [i]*arnold_race_auc['Attribute'].count())
    race_auc.append(arnold_race_auc_final)
    
    ################################
    ## condition pn
    arnold_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['arnold_nca'],
                                                                         labels = test_y)
    arnold_condition_pn_final = arnold_condition_pn.assign(fold_num = [i]*arnold_condition_pn['Attribute'].count())
    condition_pn.append(arnold_condition_pn_final)
    
    ################################
    ## no condition pn
    arnold_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['arnold_nca'],
                                                                labels = test_y)
    arnold_no_condition_pn_final = arnold_no_condition_pn.assign(fold_num = [i]*arnold_no_condition_pn['Attribute'].count())
    no_condition_pn.append(arnold_no_condition_pn_final)
    
    i += 1
    
## race_auc
arnold_race_auc = pd.concat(race_auc, ignore_index=True)
arnold_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_race_auc = arnold_race_auc.reset_index(drop=True)

## condition_pn
arnold_condition_pn = pd.concat(condition_pn, ignore_index=True)
arnold_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_condition_pn = arnold_condition_pn.reset_index(drop=True)

## no_condition_pn
arnold_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
arnold_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_no_condition_pn = arnold_no_condition_pn.reset_index(drop=True)

### Results

In [25]:
#### save results
summary_drug6_KY_interpret = {"CART": cart_summary,
                              "EBM": ebm_summary, 
                              'Lasso Stumps': stump_summary, 
                              'RiskSLIM': riskslim_summary,  
                              'Arnold PSA': arnold_auc}
%store summary_drug6_KY_interpret

Stored 'summary_drug6_KY_interpret' (dict)


In [27]:
results = [["CART", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["EBM", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["Lasso Stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['RiskSLIM', np.mean(riskslim_summary['test_auc'])],
           ['Arnold PSA', round(np.mean(arnold_auc), 3)]]
results

[['CART', 0.7694426520036945, 0.030450052937010308],
 ['EBM', 0.7767213806183678, 0.00857705708791905],
 ['Lasso Stumps', 0.7768786226998354, 0.0008030259055355815],
 ['RiskSLIM', 0.7517732771934686],
 ['Arnold PSA', 0.672]]

In [28]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc'])]

In [29]:
path = "C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY Results/Models/Six Month/"
results = [["Drug", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold_auc), 3)) + " (" + np.str(round(np.std(arnold_auc),3)) + ")"]]
with open(path + 'Interpretable Models Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### confusion matrix

In [30]:
cart_confusion = cart_summary['confusion_matrix_stats']
ebm_confusion = ebm_summary['confusion_matrix_stats']
riskslim_confusion = riskslim_summary['confusion_matrix_stats']
stumps_confusion = stump_summary['confusion_matrix_stats']

cart_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/confusion/six-month/drug/cart_confusion.csv', index=None,header=True)
ebm_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/confusion/six-month/drug/ebm_confusion.csv', index=None,header=True)
riskslim_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/confusion/six-month/drug/riskslim_confusion.csv', index=None,header=True)
stumps_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/confusion/six-month/drug/stumps_confusion.csv', index=None,header=True)

### calibration matrix

In [31]:
cart_calibration = cart_summary['calibration_stats']
ebm_calibration = ebm_summary['calibration_stats']
riskslim_calibration = riskslim_summary['calibration_stats']
stumps_calibration = stump_summary['calibration_stats']

cart_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/calibration/six-month/drug/cart_calibration.csv', index=None,header=True)
ebm_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/calibration/six-month/drug/ebm_calibration.csv', index=None,header=True)
riskslim_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/calibration/six-month/drug/riskslim_calibration.csv', index=None,header=True)
stumps_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/calibration/six-month/drug/stumps_calibration.csv', index=None,header=True)

### race auc matrix

In [32]:
cart_race_auc = cart_summary['race_auc']
ebm_race_auc = ebm_summary['race_auc']
riskslim_race_auc = riskslim_summary['race_auc']
stumps_race_auc = stump_summary['race_auc']

cart_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/six-month/drug/cart_race_auc.csv', index=None,header=True)
ebm_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/six-month/drug/ebm_race_auc.csv', index=None,header=True)
riskslim_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/six-month/drug/riskslim_race_auc.csv', index=None,header=True)
stumps_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/six-month/drug/stumps_race_auc.csv', index=None,header=True)
arnold_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/six-month/drug/arnold_race_auc.csv', index=None,header=True)

### condition pn

In [33]:
cart_condition_pn = cart_summary['condition_pn']
ebm_condition_pn = ebm_summary['condition_pn']
riskslim_condition_pn = riskslim_summary['condition_pn']
stumps_condition_pn = stump_summary['condition_pn']

cart_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/six-month/drug/cart_condition_pn.csv', index=None,header=True)
ebm_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/six-month/drug/ebm_condition_pn.csv', index=None,header=True)
riskslim_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/six-month/drug/riskslim_condition_pn.csv', index=None,header=True)
stumps_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/six-month/drug/stumps_condition_pn.csv', index=None,header=True)
arnold_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/six-month/drug/arnold_conditon_pn.csv', index=None,header=True)

### no condition pn

In [34]:
cart_no_condition_pn = cart_summary['no_condition_pn']
ebm_no_condition_pn = ebm_summary['no_condition_pn']
riskslim_no_condition_pn = riskslim_summary['no_condition_pn']
stumps_no_condition_pn = stump_summary['no_condition_pn']

cart_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/six-month/drug/cart_no_condition_pn.csv', index=None,header=True)
ebm_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/six-month/drug/ebm_no_condition_pn.csv', index=None,header=True)
riskslim_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/six-month/drug/riskslim_no_condition_pn.csv', index=None,header=True)
stumps_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/six-month/drug/stumps_no_condition_pn.csv', index=None,header=True)
arnold_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/six-month/drug/arnold_no_conditon_pn.csv', index=None,header=True)